# Exploring Ebay Car Sales Data

## 1. Introduction

In this project, we will handle a dataset comprised of information about used cars, announced on the classifieds section of the German eBay website, *eBay Kleinanzeigen*.
The original dataset, scraped and uploaded to Kaggle by user orgesleka, is no longer available, but you can now find it [here](https://data.world/data-society/used-cars-data).

The data dictionary for this dataset is the following:
- `dateCrawled`: When this ad was first crawled. All field-values are taken from this date.
- `name`: Name of the car.
- `seller`: Whether the seller is private or a dealer.
- `offerType`: The type of listing
- `price`: The price on the ad to sell the car.
- `abtest`: Whether the listing is included in an A/B test.
- `vehicleType`: The vehicle Type.
- `yearOfRegistration`: The year in which the car was first registered.
- `gearbox`: The transmission type.
- `powerPS`: The power of the car in PS.
- `model`: The car model name.
- `kilometer`: How many kilometers the car has driven.
- `monthOfRegistration`: The month in which the car was first registered.
- `fuelType`: What type of fuel the car uses.
- `brand`: The brand of the car.
- `notRepairedDamage`: If the car has a damage which is not yet repaired.
- `dateCreated`: The date on which the eBay listing was created.
- `nrOfPictures`: The number of pictures in the ad.
- `postalCode`: The postal code for the location of the vehicle.
- `lastSeenOnline`: When the crawler saw this ad last online.


## 2. Aim of the project
Clean the data and analyze the used car listing.

In [1]:
# import the pandas and NumPy libraries

import pandas as pd
import numpy as np

# read the autos.csv file into pandas, and assign it to the variable name autos
autos=pd.read_csv('autos.csv',encoding='Latin-1')

FileNotFoundError: [Errno 2] No such file or directory: 'autos.csv'

In [ ]:
autos

In [ ]:
autos.info()
autos.head()

### Observations

Right away, we can see that the following columns have missing (null) values:
- `vehicleType`
- `gearbox`
- `model`
- `fuelType`
- `notRepairedDamage`

We can also observe the columns `price` and `odometer ` are objects, which we will probably have to modify into a float type.

We can see that the names of the columns have lower- and uppercase letters and no spaces between them. This is known as [camelcase](https://en.wikipedia.org/wiki/Camel_case#:~:text=Camel%20case%20(sometimes%20stylized%20as,word%20starting%20with%20either%20case.). It will be easier for us to handle column names in the future if we convert them into [snakecase](https://en.wikipedia.org/wiki/Snake_case). It will also be helpful to rename some columns with a more descriptive title of its content (e.g. convert `price` into `price_dollars`).

## 3. Column Renaming

In [ ]:
# print an array of the existing column names

autos.columns

In [ ]:
# rename columns into snakecase

autos.rename(columns={'dateCrawled':'date_crawled', 'offerType':'offer_type', 'price':'price_dollars', 'abtest':'ab_test',
       'vehicleType':'vehicle_type', 'yearOfRegistration':'registration_year','powerPS':'power_ps', 'odometer':'odometer_km', 'monthOfRegistration':'registration_month', 'fuelType':'fuel_type',
       'notRepairedDamage':'unrepaired_damage', 'dateCreated':'ad_created', 'nrOfPictures':'nr_of_pictures', 'postalCode':'postal_code',
       'lastSeen':'last_seen'}, inplace=True)

In [ ]:
autos.head()

Now that we have converted the column names into a more managable and informative format, we will do some basic data exploration to determine what other cleaning tasks need to be done. We will start by: 

- Dropping any text columns where all or almost all values are the same, as they don't have useful information for analysis.
- Finding examples of numeric data stored as text, which can be cleaned and converted. Our initial analysis has already detected `price_dollars` and `odometer_km` as such examples.

In [ ]:
# view min/max/median/mean etc.

autos.describe(include='all')

From the table above, we can conclude:
- The `seller` and `offer_type` columns have only 2 unique values. Furthermore, the top value has a frequency of 49999, meaning that all but one row present this value.  Therefore, those columns contain virtually no useful information for analysis.
- The columns `price_dollars` and `odometer_km` are numeric data stored as text and need to be converted.
- The `nr_of_pictures` column is irrelevant to us, as all rows present only the 0.0 value.



## 4. Deleting irrelevant columns:
### 4.1 `seller`, `offer_type` and `nr_of_pictures`

In [ ]:
# look up the number of initil rows in the dataframe

print('Number of columns:',autos.shape[1])

# drop the columns

autos.drop(columns=['seller','offer_type','nr_of_pictures'], 
           inplace=True)
print('Updated number of columns:',autos.shape[1])


## 5. Text to numberic dtype conversion:
### 5.1 `price_dollars`

In [ ]:
# view min/max/median/mean etc.

autos['price_dollars'].describe()

In [ ]:
# Remove any non-numeric characters.

autos['price_dollars'] = autos['price_dollars'].str.replace(',','').str.replace('$','')                                                                                 

In [ ]:
# Convert the column to a numeric dtype.

autos['price_dollars']=autos['price_dollars'].astype('int')

In [ ]:
autos['price_dollars']

### 5.2 `odometer_km`

In [ ]:
# view min/max/median/mean etc.

autos['odometer_km'].describe()

In [ ]:
# Remove any non-numeric characters.

autos['odometer_km'] = autos['odometer_km'].str.replace(',','').str.replace('km','')                                                                                 


In [ ]:
# Convert the column to a numeric dtype.

autos['odometer_km'] = autos['odometer_km'].astype('int')

In [ ]:
autos['odometer_km']

## 6. Data exploration:

### 6.1 `price_dollars`

In [ ]:
# See how many unique values are in the column

autos['price_dollars'].unique().shape

In [ ]:
# view min/max/median/mean etc.

autos['price_dollars'].describe()

### Observations

From this, we can already tell that there are some problems with the values in `price_dollars`:
- The min value is 0 dollars, which is unusual.
- The max value is 1e+8 dollars, which is clearly incorrect.
- The mean is 9840.0 dollars standard deviation is 481104.4 dollars, which is likely caused by the outlier discussed above.

In [ ]:
# calculate the counts for each value. 

autos['price_dollars'].value_counts()

We can see that there are 1421 entries with a price of 0 dollars.

In [ ]:
# calculate the values with their respective counts in ascending order

autos['price_dollars'].value_counts().sort_index(ascending= True)

There are an additional 156 entries with a set price of 1 dollar. This could be because sellers do not wish to set a price upfront and prefer to negotiate the price of the car in private. We will remove these entries from our analysis. 

Other values that stand out are prices equal and superior to 999990 dollars. In total, there are 14 entries in this range.
Given that we are working with a dataset of used cars, these prices are unreasonably high and drive our mean and standard deviation up. Therefore, we will consider them outliers and remove them from our analysis.

In [ ]:
# remove outliers

cleaned_prices_autos=autos[autos["price_dollars"].between(2,999989)]

In [ ]:
cleaned_prices_autos.shape

In [ ]:
# view min/max/median/mean etc.

cleaned_prices_autos['price_dollars'].describe()

In [ ]:
# calculte the counts for each value

cleaned_prices_autos['price_dollars'].value_counts()

After cleaning the `price_dollars` column, we are left with **48409 entries**. Our mean price is now **5907.9 dollars**, which is lower that before, as is the standard deviation. The most frequent price for an used car is **500 dollars**, which appears in **781 entries (1.6%)**. 


### 6.2 `odometer_km`

In [ ]:
# See how many unique values are in the column

cleaned_prices_autos['odometer_km'].unique().shape

In [ ]:
# view min/max/median/mean etc.

cleaned_prices_autos['odometer_km'].describe()

In [ ]:
# calculte the counts for each value

cleaned_prices_autos['odometer_km'].value_counts()

The `odometer_km` column seems to have no major issues:
- The min value is 5000 km, present in 815 entries.
- The max value is **150000 km**, which is also the most frequent value, occuring in **31307 entries (64.7%)**.
- The mean is **125788.9 km**, with a standard deviation of      39737.8 km. 


Most entries have a value superior to 50000 km. Since we are working with used cars, this seems reasonable. 
Therefore, we will not remove any entries based on unrealistically high or low `odometer_km` values.

In [ ]:
# make a copy of the dataframe under a shorter name

clean_autos = cleaned_prices_autos.copy()

## 6.3 dates and times

There are several columns that represent date values:
- `date_crawled`: When the ad was first crawled. Added by the crawler.
- `last_seen`: When the crawler saw the ad last online. Added by the crawler
- `ad_created`: The date on which the eBay listing was created. Created by the website.
- `registration_month`: The month in which the car was first registered. Created by the website.
- `registration_year`: The year in which the car was first registered. Created by the website.


In [ ]:
clean_autos.info()

### Observations

As of now, the `date_crawled`, `ad_created` and `last_seen` columns are recognized as string types (object). The `registration_year` and `registration_month` columns, on the other hand, are stored as numeric data. 

We will convert the data stored in `date_crawled`, `ad_created` and `last_seen` into numeric data, so we can process it quantitatively.

### 6.3.1  `date_crawled`

In [ ]:
clean_autos['date_crawled'].describe()

As we can see above, the first 10 characters correspond to the date (e.g. 2016-03-12). Therefore, we can extract the date values and then generate a distribution using the `Series.value_counts()` command. 

In [ ]:
# extract the date from the string

clean_autos['date_crawled']=clean_autos['date_crawled'].str[:10]

In [ ]:
clean_autos['date_crawled'].head()

In [ ]:
clean_autos['date_crawled'].value_counts()

In [ ]:
# calculate the distribution of values in percentages instead of counts

clean_autos['date_crawled'].value_counts(normalize=True, dropna=False)


In [ ]:
# calculate the distribution of values in percentages and in ascending order

clean_autos['date_crawled'].value_counts(normalize=True, dropna=False).sort_index(ascending=True)


As we can observe above, there are 48409 entries in `date_crawled`.
The frequencies for each value are quite distributed between all the different values. The range of dates starts on **2016-03-05** and ends on **2016-04-07**.
According to this column, most ads were crawled in **2016-04-03 (3.86%)**.

### 6.3.2 `ad_created`

In [ ]:
clean_autos['ad_created'].describe()

In [ ]:
# extract the date from the string

clean_autos['ad_created']=clean_autos['ad_created'].str[:10]

In [ ]:
# calculate the distribution of values in percentages 

clean_autos['ad_created'].value_counts(normalize=True, dropna=False)


In [ ]:
# calculate the distribution of values in percentages and in ascending order

clean_autos['ad_created'].value_counts(normalize=True, dropna=False).sort_index(ascending=True)


In the `ad_created` column, there are also 48409 entries. The range of dates from the  columns starts on **2015-06-11** and ends on **2016-04-07**. According to this column, the most popular day for ad creation was **2016-04-03 (3.88%)**.


### 6.3.3 `last_seen`

In [ ]:
clean_autos['last_seen'].describe()

In [ ]:
# extract the date from the string

clean_autos['last_seen']=clean_autos['last_seen'].str[:10]

In [ ]:
# calculate the distribution of values in percentages

clean_autos['last_seen'].value_counts(normalize=True, dropna=False)


In [ ]:
# calculate the distribution of values in percentages and in ascending order

clean_autos['last_seen'].value_counts(normalize=True, dropna=False).sort_index(ascending=True)


In the `last_seen` column, there are again 48409 entries. The range of dates from the  columns starts on **2016-03-05** and ends on **2016-04-07**. According to this column, the day when most ads were last seen online is **2016-04-06 (2.21%)**.

### Date and time: summary
        

| Column     | Range of dates | Most frequent day
| ----------- | ----------- | ----------- |
| **`date_crawled`**   | 2016-03-05 to 2016-04-07   |2016-04-03 (3.86%)|
| **`ad_created`**   | 2015-06-11 to 2016-04-07     |2016-04-03 (3.88%)|
| **`last_seen`**  | 2016-03-05 to 2016-04-07       |2016-04-06 (2.21%)|

### Main conclusions:

- `dateCrawled` displays the date the ads were crawled. Ad crawling started in March of 2016 and ended in April of the same year.
- Ads created as early as June 2015 are included in this dataset, as we can see in the `ad_created` column.
- The `last_seen` column contains the date when the crawler last saw the ad online. The day when most ads were last seen online is 2016-04-06 (2.21%).


## 6.4 'registration_year'

In [ ]:
clean_autos['registration_year'].describe()

As we can see above, the `registration_year` column contains some errors. The maximum value is 9999 and the minimum is 1000, both clearly incorrect. 

Given that the `registration_year` corresponds to the year in which the car was first registered, we known that cannot be after 2015, the year the first ads were created.
As for the earliest acceptable year for car registration, we will accept any year after 1900.

In [ ]:
# remove the rows where the 'registation_year' values are outside the range we have defined

clean_autos = clean_autos[clean_autos['registration_year'].between(1900,2015)]

In [ ]:
clean_autos['registration_year'].describe()

In [ ]:
clean_autos['registration_year'].value_counts(normalize=True)

As we can see, the minimum value for this column is 1910, so cars on sale were registered as early as that. 
We can also observe that the year **2000** is the year in which the **highest number of cars were registered (6.9%)**. 

In [ ]:
# select the top 15 most frequent values 

clean_autos['registration_year'].value_counts(normalize=True, dropna=False)[0:15]

In [ ]:
# add the frequencies of the top 15 most frequent values (excluding the first)

sum(clean_autos['registration_year'].value_counts(normalize=True, dropna=False)[0:15])

In addition, almost **80%** of all car registrations occured between **1997** and **2011**. We can conclude from this analysis that, at the time of crawling, the majority of cars on sale in this platform were at least more than 5 years old. 


## 7. Determining `time_online`

Using the `last_seen` and `ad_created` columns, we can calculate how long on average an ad remains online until the car is sold. We will name this new variable `time_online`.


In [ ]:
# convert 'ad_created' column to datetime

clean_autos['ad_created']=pd.to_datetime(clean_autos['ad_created'])

In [ ]:
# convert 'last_seen' column to datetime

clean_autos['last_seen']=pd.to_datetime(clean_autos['last_seen'])

In [ ]:
clean_autos['ad_created'].describe()

In [ ]:
clean_autos['last_seen'].describe()

In [ ]:
# create a new column with the time past between `ad_created` and `last_seen``

clean_autos['time_online']=clean_autos['last_seen']-clean_autos['ad_created']

In [ ]:
clean_autos['time_online'].shape

In [ ]:
# calculate the distribution of values in percentages 

clean_autos['time_online'].value_counts(normalize=True, dropna=False)

In [ ]:
# calculate the distribution of values in percentages and in ascending order

clean_autos['time_online'].value_counts(normalize=True, dropna=False).sort_index(ascending=True)

According to this result, **13.9%** of ads were online for **less than a day**. It is likely that these ads were removed by the poster not because the car was sold, but because of some error in the description or price or something of that nature. 

In [ ]:
# select the top 15 most frequent values (excluding the first)

clean_autos['time_online'].value_counts(normalize=True, dropna=False)[1:15]

In [ ]:
# add the frequencies of the top 15 most frequent values (excluding the first)

sum(clean_autos['time_online'].value_counts(normalize=True, dropna=False).sort_index(ascending=True)[1:15])

### Main conclusions:

If we exclude the '0 days' value from the top 15 most frequent results, we obtain that **62.8%** of the ads are removed up to **14 days** after their creation. We can infer from this that most of the used cars announced in the  *eBay Kleinanzeigen* platform are sold after less than 2 weeks after publishing. 

## 8. Exploring mean price by brand

In [ ]:
clean_autos['brand'].describe()

In [ ]:
clean_autos['brand'].value_counts()

In [ ]:
clean_autos['brand'].value_counts(normalize=True)

We will start by selecting the top 10 most common brands. 

In [ ]:
# select the top 10 brands

top_ten_brands=clean_autos['brand'].value_counts().index[0:10]
print(top_ten_brands)

In [ ]:
# create an empty dictionary to hold the mean price for each brand

mean_price_brand={}

# loop over each top 10 brand and retrieve the mean price

for b in top_ten_brands:
    brand_rows=clean_autos[clean_autos['brand']==b]
    mean_price=brand_rows['price_dollars'].mean()
    mean_price_brand[b]=mean_price

In [ ]:
mean_price_brand

### Main conclusions:

Among the top 10 most common brands, there are 3 **luxury brands**: `audi`, `mercedes_benz` and `bmw`. Used cars on sale by these brands have higher prices, costing on average around **9000 dollars**. 

The remaining brands are more affordable, and their mean prices range between 2500 to 5500 dollars. 

`fiat` and `renault` are the **cheapest brands** on average, with prices below **3000 dollars.** `volkswagen` and `seat` are **middle-range brands**, with mean prices closer to **5000 dollars**. 

## 9. Exploring mean price and average mileage 

In [ ]:
# create an empty dictionary to hold the mean mileage for each brand

mean_miles_brand={}

# loop over each top 10 brand and retrieve the mean price

for b in top_ten_brands:
    brand_rows=clean_autos[clean_autos['brand']==b]
    mean_miles=brand_rows['odometer_km'].mean()
    mean_miles_brand[b]=mean_miles

In [ ]:
mean_miles_brand

In [ ]:
# convert the mean prices dictionary to a series object, using the series constructor

price_series=pd.Series(data=mean_price_brand)

In [ ]:
# convert the mean mileage dictionary to a series object, using the series constructor

miles_series=pd.Series(data=mean_miles_brand)

In [ ]:
price_series

In [ ]:
# create a dataframe from the price series using the dataframe constructor

price_miles_dataframe=pd.DataFrame(price_series, columns=['mean_price'])

In [ ]:
price_miles_dataframe

In [ ]:
# add the mileage series as a new column in the new dataframe

price_miles_dataframe['mean_mileage']=miles_series

In [ ]:
price_miles_dataframe

In [ ]:
price_miles_dataframe['mean_mileage'].describe()

### Main conclusions:

- There are no drastic differences in the average mileage between the top 10 brands. All brands display a **mean mileage over 110000 km**, and the standard deviation of the series is below 5000 km.
- The brand with the highest average mileage is `mercedes_benz` (**132446.4 km**) and the one with the lowest is `fiat` (**116589.9 km**). The 15856.5 km difference does not seem to justify the difference in mean prices, considering that `mercedes_benz` cars are 204.9% more expensive on average than `fiat` cars.
- Similarly, we can see that `opel` cars have almost exactly the same mean mileage as `audi` cars, while their average price is 67.7% cheaper.
- Therefore, we can conclude that the **average mileage of cars produced by the top 10 brands is not likely to influence the cars' average price**.

## 10. Final summary

We performed several data cleaning tasks before analyzing the dataset, such as:
- removing 3 irrelevant columns (`seller`, `offer_type`, and `nr_of_pictures`) that added no valuable information;
- excluding unreasonable entries in the `odometer_km`, `price_dollars` and `registration_year` columns, which we categorized as outliers.
    

Regarding the analysis of the data, we have determined that:
- at the time of crawling, the majority of cars on sale in the *eBay Kleinanzeigen* platform were at least more than 5 years old;
- most of the used cars announced in the crawled ads were sold after less than 2 weeks after publishing;
- Among the top 10 most common brands, there were 3 luxury brands (`audi`, `mercedes_benz` and `bmw`, average price ~9000 dollars), middle-range brands (`volkswagen` and `seat`, average price ~5000 dollars) and cheaper brands (`fiat` and `renault`, average price ~3000 dollars);
- The mean mileage of cars produced by the top 10 brands does not strongly correlate with the average price and it is unlikely to influence the cars' price.
